In [ ]:
from bs4 import BeautifulSoup  # для парсинга HTML и XML
from bs4.dammit import EncodingDetector  # для определения кодировки документа
from newspaper import Article  # для извлечения новостных статей
import requests  # для отправки HTTP-запросов
import psycopg2  # для работы с базами данных PostgreSQL

def entrance(user, password):
    try:
        # Устанавливаем соединение с базой данных PostgreSQL
        conn = psycopg2.connect(
            database="business_Intelligence",   # Указываем название базы данных
            user=user,   # Указываем имя пользователя, под которым производится подключение
            password=password,  # Указываем пароль пользователя
            host="localhost",   # Указываем хост (обычно это localhost)
            port="5432"   # Указываем порт (обычно это 5432)
        )

        # Создаем курсор для выполнения SQL-запросов
        cursor = conn.cursor()

        # Если подключение удалось, выводим сообщение об успешном подключении
        print("Подключение удалось")

        # Возвращаем объект курсора и объект соединения
        return cursor, conn

    except Exception as e:
        # Если возникла ошибка подключения, выводим сообщение об ошибке
        print("Ошибка подключения")
        print(f"Ошибка: {e}")

        # Возвращаем None для курсора и 0 для соединения, чтобы сигнализировать об ошибке
        return None, 0

def parser_news(words, URL):
    # Разделяем строку слов на список с помощью разделителя ","
    words = words.split(',')
    # Создаем пустой список для статей
    art = []
    # Итерируемся по каждому URL-адресу из списка URL
    for url in URL:
        # Указываем парсер для BeautifulSoup
        parser = 'html.parser'
        # Задаем заголовок для запроса
        headers = {"User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:89.0) Gecko/20100101 Firefox/89.0"}

        try:
            # Отправляем GET-запрос на URL-адрес
            resp = requests.get(url, headers=headers)
        except requests.exceptions.ConnectionError:
            # Если возникает ошибка соединения, выводим сообщение об ошибке и переходим к следующей итерации цикла
            print("Ошибка соединения с URL:", url)
            continue

        # Получаем кодировку ответа от сервера
        http_encoding = resp.encoding if 'charset' in resp.headers.get('content-type', '').lower() else None
        # Получаем кодировку HTML-страницы при помощи кодировщика
        html_encoding = EncodingDetector.find_declared_encoding(resp.content, is_html=True)
        # Выбираем кодировку, используя закодированную HTML-страницу или кодировку ответа от сервера
        encoding = html_encoding or http_encoding
        # Обрабатываем HTML-страницу при помощи BeautifulSoup
        soup = BeautifulSoup(resp.content, parser, from_encoding=encoding)

        # Ищем все внутренние ссылки на странице
        links = []
        for link in soup.find_all('a', href=True):
            # Пропускаем ссылки на JavaScript
            if "javascript" in link["href"]:
                continue
            links.append(link['href'])

        # Итерируемся по каждой ссылке на странице
        for link in links:
            try:
                # Загружаем статью из ссылки
                article = Article(link)
                # Загружаем статью с помощью методов .download() и .parse()
                article.download()
                article.parse()
                # Ищем каждое заданное слово в тексте статьи
                for word in words:
                    if word in article.text:
                        # Если слово найдено, добавляем статью в список статей "art"
                        art.append(article.text)
            except:
                # Если возникает ошибка при загрузке статьи, переходим к следующей итерации цикла
                pass

    # Возвращаем список статей, содержащих заданные слова
    return art

def get_media_urls(cursor):
    # Выполняем SQL-запрос для выборки всех URL-адресов из таблицы "media"
    cursor.execute("SELECT url FROM public.media")
    # Получаем все строки результата запроса в виде списка кортежей и извлекаем из них URL-адреса
    urls = [row[0] for row in cursor.fetchall()]
    # Возвращаем список URL-адресов
    return urls

def insert_company(cur):
    # Запрашиваем у пользователя данные о компании
    company_name = input("Введите название компании:")
    company_description = input("Введите описание компании:")
    industry = input("Введите отрасль:")
    year_established = input("Введите год основания:")
    number_of_employees = int(input("Введите количество сотрудников:"))
    location_country = input("Введите страну:")
    location_city = input("Введите город:")
    address = input("Введите адрес:")
    products_services = input("Введите продукт/услугу:")
    competitors = input("Введите конкурентов:")
    growth_trends_forecasting = input("Введите тенденцию:")
    
    try:
        # Вызываем хранимую процедуру "insert_company" и передаем ей параметры
        cur.execute('CALL public.insert_company(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);', (
            company_name, 
            company_description, 
            industry,
            year_established,
            number_of_employees, 
            location_country, 
            location_city, 
            address, 
            products_services, 
            competitors, 
            growth_trends_forecasting
        ))
        
        # Выводим сообщение об успешном добавлении компании
        print("Компания успешно добавлена в базу данных!")
    
    except Exception as e:
        # Выводим сообщение об ошибке в случае неудачной попытки добавить компанию в таблицу
        print("Ошибка ввода в таблицу:", e)

def insert_company_finance(cur):
    # Запрашиваем у пользователя данные о компании
    company_name = input("Введите название компании:")
    revenue = int(input("Введите доход:"))
    profit = int(input("Введите прибыль:"))
    assets = int(input("Введите автивы:"))
    investments = int(input("Введите инвестиции:"))
    dividends = int(input("Введите дивиденты:"))
    
    try:
        # Вызываем хранимую процедуру "insert_company_finance" и передаем ей параметры
        cur.execute('CALL public.insert_company_finance(%s, %s, %s, %s, %s, %s);', (
            company_name, 
            revenue,
            profit, 
            assets,
            investments, 
            dividends
        ))
        
        # Выводим сообщение об успешном добавлении компании
        print("Компания успешно добавлена в базу данных!")
    
    except Exception as e:
        # Выводим сообщение об ошибке в случае неудачной попытки добавить компанию в таблицу
        print("Ошибка ввода в таблицу:", e)

def insert_contact(cur):
    # Запрашиваем у пользователя данные о контакте
    first_name = input("Введите имя:")
    last_name = input("Введите фамилию:")
    position = input("Введите должность:")
    company_name = input("Введите компанию:")
    phone_number = input("Введите номер телефона:")
    email = input("Введите электронную почту:")
    
    try:
        # Вызываем хранимую процедуру "sp_insert_contact" и передаем ей параметры
        cur.execute("CALL public.sp_insert_contact(%s, %s, %s, %s, %s, %s);", (first_name, last_name, position,company_name, phone_number, email))
        # Выводим сообщение об успешном добавлении контакта
        print("Контакт успешно добавлен в базу данных!")
    except Exception as e:
        # Выводим сообщение об ошибке в случае неудачной попытки добавить контакт в таблицу
        print("Ошибка ввода в таблицу:", e)

def insert_country(cur):
    # Запрашиваем у пользователя данные о стране
    country_name = input("Введите название страны:")
    population = int(input("Введите численность населения:"))
    political_system = input("Введите политическая система:")
    legal_system = input("Введите юридическая система:")
    stability = input("Введите стабильность:")
    relations_with_other_countries = input("Введите отношения с другими странами:")
    industry = input("Введите индустрию:")
    
    try:
        # Вызываем хранимую процедуру "insert_country" и передаем ей параметры
        cur.execute("CALL public.insert_country(%s, %s, %s, %s, %s, %s, %s)", 
            ( country_name, population, political_system, legal_system, stability, relations_with_other_countries, industry)
        )
        # Выводим сообщение об успешном добавлении страны
        print("Страна успешно добавлена в базу данных!")
        conn.commit() # применение изменений в базе данных
    except Exception as e:
        # Выводим сообщение об ошибке в случае неудачной попытки добавить страну в таблицу
        print("Ошибка ввода в таблицу:", e)

def insert_country_numeric(cur):
    # Запрашиваем у пользователя данные о стране
    country_name = input("Введите название страны:")
    gdp = float(input("Введите ВВП:"))
    gdp_growth = float(input("Введите рост ВВП:"))
    inflation = float(input("Введите инфляцию:"))
    unemployment = float(input("Введите безработицу:"))
    investments = float(input("Введите инвестиции:"))
    
    try:
        # Вызываем хранимую процедуру "insert_country_numeric" и передаем ей параметры
        cur.execute("CALL public.insert_country_numeric(%s, %s, %s, %s, %s, %s)", 
            ( country_name, gdp, gdp_growth, inflation, unemployment, investments)
        )
        # Выводим сообщение об успешном добавлении страны
        print("Страна успешно добавлена в базу данных!")
        conn.commit() # применение изменений в базе данных
    except Exception as e:
        # Выводим сообщение об ошибке в случае неудачной попытки добавить страну в таблицу
        print("Ошибка ввода в таблицу:", e)

def insert_media(cur):
    # Запрашиваем у пользователя данные о медиа-источнике
    name = input("Введите название медиа-источника:")
    url = input("Введите ссылку:")
    
    try:
        # Вызываем хранимую процедуру "sp_insert_media" и передаем ей параметры
        cur.execute("CALL public.sp_insert_media(%s, %s)", (
            name,
            url
        ))
        # Выводим сообщение об успешном добавлении медиа-источника
        print("Медиа-источник успешно добавлена в базу данных!")
    except Exception as e:
        # Выводим сообщение об ошибке в случае неудачной попытки добавить медиа-источник в таблицу
        print("Ошибка ввода в таблицу:", e)

def delete_company(cur):
    # Запрашиваем у пользователя название компании, которую нужно удалить
    company_name = input("Введите название компании:")
    
    try:
        # Вызываем хранимую процедуру "sp_delete_company" и передаем ей параметры
        cur.execute("CALL public.sp_delete_company(%s)", (company_name, ))
        # Проверяем, была ли удалена какая-либо запись из таблицы
        if cur.rowcount == 0:
            # Выводим сообщение, если данная компания не найдена в таблице
            print("Компания с таким названием не найдена!")
        else:
            # Если удаление прошло успешно, выводим соответствующее сообщение
            print("Компания успешно удалена из базы данных!")
    except Exception as e:
        # Выводим сообщение об ошибке в случае неудачной попытки удаления компании из таблицы
        print("Ошибка удаления данных в таблице:", e)

def delete_contact(cur):
    # Запрашиваем у пользователя имя и фамилию контакта, которого нужно удалить
    first_name = input("Введите имя:")
    last_name = input("Введите фамилию:")
    
    try:
        # Вызываем хранимую процедуру "sp_delete_contact" и передаем ей параметры
        cur.execute("CALL public.sp_delete_contact(%s,%s)", (first_name, last_name, ))
        # Проверяем, была ли удалена какая-либо запись из таблицы
        if cur.rowcount == 0:
            # Выводим сообщение, если контакт не найден в таблице
            print("Контакт с таким именем и фамилией не найден!")
        else:
            # Если удаление прошло успешно, выводим соответствующее сообщение
            print("Контакт успешно удален из базы данных!")
    except Exception as e:
        # Выводим сообщение об ошибке в случае неудачной попытки удаления контакта из таблицы
        print("Ошибка удаления данных в таблице:", e)

def delete_country(cur):
    # Запрашиваем у пользователя название страны, которую нужно удалить
    country_name = input("Введите название страны:")
    
    try:
        # Вызываем хранимую процедуру "sp_delete_country" и передаем ей параметры
        cur.execute("CALL public.sp_delete_country(%s)", (country_name, ))
        # Проверяем, была ли удалена какая-либо запись из таблицы
        if cur.rowcount == 0:
            # Выводим сообщение, если страна не найдена в таблице
            print("Страна с таким названием не найдена!")
        else:
            # Если удаление прошло успешно, выводим соответствующее сообщение
            print("Страна успешно удалена из базы данных!")
    except Exception as e:
        # Выводим сообщение об ошибке в случае неудачной попытки удаления страны из таблицы
        print("Ошибка удаления данных в таблице:", e)

def delete_media(cur):
    # Запрашиваем у пользователя название медиа-ресурса, который нужно удалить
    name = input("Введите название медиа-источника:")
    
    try:
        # Вызываем хранимую процедуру "sp_delete_media" и передаем ей параметры
        cur.execute("CALL public.sp_delete_media(%s)", (name, ))
        # Проверяем, была ли удалена какая-либо запись из таблицы
        if cur.rowcount == 0:
            # Выводим сообщение, если медиа-источник не найден в таблице
            print("Медиа-источник с таким названием не найден!")
        else:
            # Если удаление прошло успешно, выводим соответствующее сообщение
            print("Медиа-источник успешно удален из базы данных!")
    except Exception as e:
        # Выводим сообщение об ошибке в случае неудачной попытки удаления медиа-ресурса из таблицы
        print("Ошибка удаления данных в таблице:", e)

def sql(cur):
    # Запрашиваем у пользователя SQL-запрос
    q = input("Введите SQL-запрос:")
    
    try:
        # Получаем текущие количество ролей в базе данных
        cur.execute("SELECT count(*) FROM pg_roles;")
        roles = cur.fetchone()[0]
        
        # Получаем текущее количество таблиц в базе данных
        cur.execute("SELECT count(*) FROM information_schema.tables WHERE table_schema = 'public';")
        tables = cur.fetchone()[0]
        
        # Выполняем SQL-запрос
        cur.execute(q)
        rows = cur.fetchall()

        # Проверяем изменение количества ролей в базе данных
        cur.execute("SELECT count(*) FROM pg_roles;")
        roles_count = cur.fetchone()[0]
        if roles_count > roles:
            cur.close()
            conn.close()
            raise Exception("Доступ временно заблокирован! Количество ролей в базе данных превысило допустимый предел.")

        # Проверяем изменение количества таблиц в базе данных
        cur.execute("SELECT count(*) FROM information_schema.tables WHERE table_schema = 'public';")
        tables_count = cur.fetchone()[0]
        if tables_count > tables:
            cur.close()
            conn.close()
            raise Exception("Доступ временно заблокирован! Количество таблиц в базе данных превысило допустимый предел.")

        # Выводим результаты запроса
        if cur.rowcount == 0:
            print("Нет данных, удовлетворяющих запросу!")
        else:
            for row in rows:
                print(row)

    except Exception as e:
        # Выводим сообщение об ошибке в случае неудачного выполнения запроса
        print("Ошибка выполнения SQL-запроса:", e)
        # Закрываем соединение с базой данных и завершаем работу программы
        cur.close()
        conn.close()
        exit()

def select_country(cur):
    # Запрашиваем у пользователя название страны
    name = input("Введите название страны:")
    
    # Формируем SQL-запрос для выборки информации о стране
    q = f"SELECT l.*, r.* FROM public.country AS l JOIN public.country_numeric AS r ON r.country_name = l.country_name WHERE l.country_name = '{name}';"

    # Выполняем SQL-запрос
    cur.execute(q)
    
    # Получаем результаты запроса
    res = cur.fetchall()
    
    # Выводим результаты запроса
    print(res)

def select_company(cur):
    # Запрашиваем у пользователя название компании
    name = input("Введите название компании:")

    # Формируем SQL-запрос для выборки информации о компании
    q = f"SELECT l.*, r.* FROM public.company AS l JOIN public.company_finance AS r ON r.company_name = l.company_name WHERE l.company_name = '{name}';"

    # Выполняем SQL-запрос
    cur.execute(q)

    # Получаем результаты запроса
    res = cur.fetchall()

    # Выводим результаты запроса
    print(res)

def select_contact(cur):
    # Запрашиваем у пользователя имя и фамилию контакта
    first_name = input("Введите имя:")
    last_name = input("Введите фамилию:")
    
    # Формируем SQL-запрос для выборки информации о контакте
    q = f"SELECT * FROM public.contact WHERE first_name = '{first_name}' AND last_name ='{last_name}'"
    
    # Выполняем SQL-запрос
    cur.execute(q)
    
    # Получаем результаты запроса
    res = cur.fetchall()
    
    # Выводим результаты запроса
    print(res)

def select_media(cur):
    # Запрашиваем у пользователя название медиа-источника
    name = input("Введите название медиа-источника:")

    # Формируем SQL-запрос для выборки информации о медиа-источнике
    q = f"SELECT * FROM public.media WHERE media_name = '{name}'"

    # Выполняем SQL-запрос
    cur.execute(q)

    # Получаем результаты запроса
    res = cur.fetchall()

    # Выводим результаты запроса
    print(res)

while True:
    # Флаг для контроля успешности выполнения входа в систему
    tr = True
    # Запрашиваем данные для входа
    print("Введите данные для входа")
    user = input("Введите логин:")
    password = input("Введите пароль:")

    # Выполняем вход в систему
    cursor, conn = entrance(user, password)
    # Если вход не выполнен, устанавливаем флаг в False
    if cursor == None:
        tr = False

    # Запускаем цикл для выполнения действий в выбранной роли пользователя
while True:
    # Флаг для контроля успешности выполнения входа в систему
    tr = True
    # Запрашиваем данные для входа
    print("Введите данные для входа")
    user = input("Введите логин:")
    password = input("Введите пароль:")

    # Выполняем вход в систему
    cursor, conn = entrance(user, password)
    # Если вход не выполнен, устанавливаем флаг в False
    if cursor == None:
        tr = False

    # Запускаем цикл для выполнения действий в выбранной роли пользователя
    while tr:
        if user == 'marketer':
            # Выводим меню для менеджера
            print("Выберите действие:")
            print("1.Получить новости из избранных источников")
            print("2.Вывести данные о компании")
            print("3.Вывести данные о стране")
            print("4.Вывести данные о медиа-источнике")
            print("100.Ввести свой запрос")

            # Ждем ввода действия
            inp = int(input())

            # Обрабатываем ввод пользователя
            if inp == 0: 
                break
            elif inp == 1:
                news = parser_news(words=input("Введите ключевые слова:"),URL=get_media_urls(cursor=cursor))
                for n in news:
                    print(n)
            elif inp == 2:
                select_company(cur=cursor)
            elif inp == 3:
                select_country(cur=cursor)
            elif inp == 4:
                select_media(cur=cursor)
            elif inp == 100:
                sql(cur=cursor)

        if user == 'analyst' or user == 'manager' or user == 'postgres':
            # Выводим меню для аналитика или администратора
            print("Выберите действие:")
            print("1.Получить новости из избранных источников")
            print("2.Внести данные о компании в БД")
            print("3.Внести финаносовые изменения в компании в БД")
            print("4.Внести данные о стране в БД")
            print("5.Внести цифровые изменения в стране в БД")
            print("6.Внести данные о человеке в БД")
            print("7.Внести данные о медиа-источнике в БД")
            print("8.Удалить данные о компании в БД")
            print("9.Удалить данные о стране в БД")
            print("10.Удалить данныe о человеке в БД")
            print("11.Удалить данные о медиа-источнике в БД")
            print("12.Вывести данные о компании")
            print("13.Вывести данные о стране")
            print("14.Вывести данные о человеке")
            print("15.Вывести данные о медиа-источнике")
            print("100.Ввести свой запрос")
            print("0.Выйти")

            # Ждем ввода действия
            inp = int(input())

            # Обрабатываем ввод пользователя
            if inp == 0:
                break
            elif inp == 1:
                news = parser_news(words=input("Введите ключевые слова:"),URL=get_media_urls(cursor=cursor))
                for n in news:
                    print(n)
            elif inp == 2:
                insert_company(cur=cursor)
            elif inp == 3:
                insert_company_finance(cur=cursor)
            elif inp == 4:
                insert_country(cur=cursor)
            elif inp == 5:
                insert_country_numeric(cur=cursor)
            elif inp == 6:
                insert_contact(cur=cursor)
            elif inp == 7:
                insert_media(cur=cursor)
            elif inp == 8:
                delete_company(cur=cursor)
            elif inp == 9:
                delete_country(cur=cursor)
            elif inp == 10:
                delete_contact(cur=cursor)
            elif inp == 11:
                delete_media(cur=cursor)
            elif inp == 12:
                select_company(cur=cursor)
            elif inp == 13:
                select_country(cur=cursor)
            elif inp == 14:
                select_contact(cur=cursor)
            elif inp == 15:
                select_media(cur=cursor)
            elif inp == 100:
                sql(cur=cursor)
            else:
                print("Ошибка ввода")

        # Фиксируем изменения в БД
        conn.commit()

    # Закрываем соединение и курсор
    if cursor == None: 
        break
    elif inp == 0:
        cursor.close()
        conn.close()
        break  
